In [1]:
#! /usr/bin/env python

import pickle
import pandas as pd
import re

import tensorflow as tf
import numpy as np
import os
import time
import datetime
import data_helpers
import text_cnn; reload(text_cnn);
from text_cnn import TextCNN
from tensorflow.contrib import learn

# Parameters
# ==================================================

# Data loading params
tf.flags.DEFINE_string("data_path","csv","Path to extracted books csv files")
tf.flags.DEFINE_float("dev_sample_percentage", .01, "Percentage of the training data to use for validation")
tf.flags.DEFINE_float("test_sample_percentage",0.4,"Percentage to leave for testing")
tf.flags.DEFINE_integer('max_sentence_length',300,'Maximum length of sentences')
tf.flags.DEFINE_string("class_encoding","full","book review data has 5 classes, can either split into positive or negative or leave the all 5")

# Model Hyperparameters
tf.flags.DEFINE_integer("embedding_dim", 300, "Dimensionality of character embedding (default: 128)")
tf.flags.DEFINE_string("filter_sizes", "3,4,5", "Comma-separated filter sizes (default: '3,4,5')")
tf.flags.DEFINE_integer("num_filters", 128, "Number of filters per filter size (default: 128)")
tf.flags.DEFINE_float("dropout_keep_prob", 0.8, "Dropout keep probability (default: 0.8)")

# Training parameters
tf.flags.DEFINE_integer("batch_size", 32, "Batch Size (default: 64)")
tf.flags.DEFINE_integer("num_epochs", 200, "Number of training epochs (default: 200)")
tf.flags.DEFINE_integer("evaluate_every", 100, "Evaluate model on dev set after this many steps (default: 100)")
tf.flags.DEFINE_integer("checkpoint_every", 100, "Save model after this many steps (default: 100)")
tf.flags.DEFINE_integer("num_checkpoints", 5, "Number of checkpoints to store (default: 5)")
# Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")

FLAGS = tf.flags.FLAGS
FLAGS._parse_flags()
print("\nParameters:")
for attr, value in sorted(FLAGS.__flags.items()):
    print("{}={}".format(attr.upper(), value))
print("")


# Data Preparation
# ==================================================
# Load data
print("Loading data...")
# Load the csv
df = data_helpers.load_data(path=FLAGS.data_path,max_sentence_length=FLAGS.max_sentence_length,encoding=FLAGS.class_encoding)
df = df.iloc[int(-FLAGS.test_sample_percentage*len(df)):]


Parameters:
ALLOW_SOFT_PLACEMENT=True
BATCH_SIZE=32
CHECKPOINT_EVERY=100
CLASS_ENCODING=full
DATA_PATH=csv
DEV_SAMPLE_PERCENTAGE=0.01
DROPOUT_KEEP_PROB=0.8
EMBEDDING_DIM=300
EVALUATE_EVERY=100
FILTER_SIZES=3,4,5
LOG_DEVICE_PLACEMENT=False
MAX_SENTENCE_LENGTH=300
NUM_CHECKPOINTS=5
NUM_EPOCHS=200
NUM_FILTERS=128
TEST_SAMPLE_PERCENTAGE=0.4

Loading data...


27040